In [304]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

import re
import functools
import nltk
from nltk.corpus import stopwords

import pymorphy2
morph = pymorphy2.MorphAnalyzer()

def clear_string(word):
    return re.sub(r'[^\w\s]', '', word)

def string_to_trigramm(string):
    for item in string.split():
        for i in range(len(item)-2):
            yield item[i:i+3]

def map_to_normal_form(string):
    for item in string.split():
        yield morph.parse(item)[0].normal_form
        
def map_to_POS(string):
    for item in string.split():
        yield morph.parse(item)[0].tag.POS
        
def convert_to_array(dataset):
    new_arr = None
    for r in dataset:
        row = []
        for column in r:
            row.append(column if '|S' not in str(column.dtype) else column.decode('UTF-8'))
            
        if new_arr is not None:
            new_arr = np.vstack((new_arr, row))
        else:
            new_arr = np.array([row])
    return new_arr

In [194]:
def get_data(filename, dtypes=[('id','i8'),('text','|S10000'),('label','i8')]):
    return np.genfromtxt(
        filename, delimiter=',', skip_header=1, 
        dtype=dtypes, names=[i[0] for i in dtypes], comments=False)

train_dataset = get_data('./dataset-3/train.csv')
test_dataset = get_data('./dataset-3/test_without_labels.csv', dtypes=[('id','i8'),('text','|S10000')])

In [346]:
# приводим к массиву, так, чтобы удобнее работать было
train_dataset_as_array = 
convert_to_array(train_dataset)
test_dataset_as_array = convert_to_array(test_dataset)

In [347]:
# чистим от мусора
vfunc = np.vectorize(lambda x: clear_string(x))
train_dataset_as_array[:,1] = vfunc(train_dataset_as_array[:,1])

In [348]:
# чистим от всяких предлогов и односложных букв
sw = stopwords.words("russian")
vfunc = np.vectorize(lambda x: ' '.join([word for word in x.split() if word not in sw]))
train_dataset_as_array[:,1] = vfunc(train_dataset_as_array[:,1])

In [349]:
# приводим к нормальной форме
vfunc = np.vectorize(lambda x: ' '.join([nr for nr in map_to_normal_form(x)]))
train_dataset_as_array[:,1] = vfunc(train_dataset_as_array[:,1])

In [352]:
print(train_dataset_as_array[:,1])

[ 'целовать стена закачаться проплывать буй жаркое объятие стон небо маленький счастие на душа слеза я уйти утро скрипнуть кровать промелькнуть чудо ты остаться спать любовь течь полезвнож возможно бритва любить кураж закон люба любовный битва там весь правило искусство фронт сходиться несмело свой проверять чувство а доверять тело ну тот'
 'рываета мы свой мир исходить слово бенуа сделать вывод исследовать чужой незнакомый мир художник делать свой терентiть конечно настоящий мастер расширять мир дополнять вскрывать отождествляться так вживаться передать состояние пространство настроение точно возможность объективно вопрос терентiть да зад'
 'ся оправдать ир или позвонить работа сказать поехать нотариус довнести картошка почистить ир повозмущаться сдаться весь равно вылечить произнести негромко он повесить трубка быстро собраться по дорога перебирать карман купюра вспомнить пумка фотографировать егор весной начало май быть холодный у должный сохраниться фотография'
 ...,
 'ать вместе с

In [353]:
# делаем триграммы
vfunc = np.vectorize(lambda x: ' '.join([nr for nr in string_to_trigramm(x)]))
ngrams = np.array([vfunc(train_dataset_as_array[:,1])])

In [354]:
print(ngrams)
#print(train_dataset_as_array[:,1])
#print(train_dataset_as_array[:,1].shape)
#print(train_dataset['label'].shape)

[[ 'цел ело лов ова ват ать сте тен ена зак ака кач ача чат ать тьс ься про роп опл плы лыв ыва ват ать буй жар арк рко кое объ бъя ъят яти тие сто тон неб ебо мал але лен ень ньк ьки кий сча час аст сти тие душ уша сле лез еза уйт йти утр тро скр кри рип ипн пну нут уть кро ров ова ват ать про ром оме мел ель льк ькн кну нут уть чуд удо ост ста тат ать тьс ься спа пат ать люб юбо бов овь теч ечь пол оле лез езв звн вно нож воз озм змо мож ожн жно бри рит итв тва люб юби бит ить кур ура раж зак ако кон люб юба люб юбо бов овн вны ный бит итв тва там вес есь пра рав ави вил ило иск ску кус усс сст ств тво фро рон онт схо ход оди дит ить тьс ься нес есм сме мел ело сво вой про ров ове вер еря рят ять чув увс вст ств тво дов ове вер еря рят ять тел ело тот'
  'рыв ыва вае ает ета сво вой мир исх схо ход оди дит ить сло лов ово бен ену нуа сде дел ела лат ать выв ыво вод исс ссл сле лед едо дов ова ват ать чуж ужо жой нез езн зна нак ако ком омы мый мир худ удо дож ожн жни ник дел ела лат 

In [380]:
tfidf_vectorizer = TfidfVectorizer()
X_tfidf = tfidf_vectorizer.fit_transform(train_dataset_as_array[:,1])

X_train, X_test, Y_train, Y_test = train_test_split(X_tfidf, train_dataset['label'], test_size=0.5, random_state=17)

In [366]:
print(X_test.shape)

(6972, 48246)


In [381]:
# байес по тестовой выборке
clf_bnb = BernoulliNB()
clf_bnb.fit(X_train, Y_train)
Y_pred = clf_bnb.predict(X_test)
Y_pred_proba = clf_bnb.predict_proba(X_test)
print('roc_auc', roc_auc_score(y_true=Y_test, y_score=Y_pred_proba[:,1]))
print(classification_report(y_true=Y_test, y_pred=Y_pred))

roc_auc 0.4562028311
             precision    recall  f1-score   support

          0       0.90      1.00      0.95      6298
          1       0.00      0.00      0.00       674

avg / total       0.82      0.90      0.86      6972



/Users/vitaly/.virtualenvs/ml-env/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [555]:
# байес по трэйн выборке
clf_bnb = BernoulliNB()
clf_bnb.fit(X_tfidf, train_dataset_as_array[:,2])
Y_pred = clf_bnb.predict(X_train)
Y_pred_proba = clf_bnb.predict_proba(X_train)
print('roc_auc', roc_auc_score(y_true=Y_train, y_score=Y_pred_proba[:,1]))
print(classification_report(y_true=Y_train, y_pred=Y_pred))    

roc_auc 0.919203093383
             precision    recall  f1-score   support

          0       0.91      1.00      0.95      6316
          1       0.00      0.00      0.00       656

avg / total       0.82      0.91      0.86      6972



/Users/vitaly/.virtualenvs/ml-env/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [ ]:
print(Y_train)

In [557]:
# тут мы выпиливаем всякий хлам, который не смогли предсказать
def find_allow(Y_train, Y_pred):
    def find_depricated(train, predict):
        dep = []
        for i in enumerate(train):
            if int(i[1]) != predict[i[0]]:
                dep.append(i[0])
        return dep

    dep = np.array( find_depricated(Y_train, Y_pred) )
    return np.array(list(filter(lambda x: x[0] not in dep, enumerate(Y_train))))[:,0]
    
    
allow_indexes = find_allow(Y_train, Y_pred)
print( allow_indexes.shape )

(6316,)


In [552]:
# байес по тестовой выборке
clf_bnb = BernoulliNB()
clf_bnb.fit(X_train[allow_indexes,], Y_train[allow_indexes])
Y_pred = clf_bnb.predict(X_test)
Y_pred_proba = clf_bnb.predict_proba(X_test)
print('roc_auc', roc_auc_score(y_true=Y_test, y_score=Y_pred_proba[:,1]))
print(classification_report(y_true=Y_test, y_pred=Y_pred))

roc_auc 0.5
             precision    recall  f1-score   support

          0       0.90      1.00      0.95      6298
          1       0.00      0.00      0.00       674

avg / total       0.82      0.90      0.86      6972



/Users/vitaly/.virtualenvs/ml-env/lib/python3.6/site-packages/sklearn/naive_bayes.py:461: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
/Users/vitaly/.virtualenvs/ml-env/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [551]:
iverted_vocab = {_id:w for (w,_id) in tfidf_vectorizer.vocabulary_.items()}
for _id in clf_bnb.coef_.argsort()[0][-150:]:
    print(iverted_vocab[_id], clf_bnb.coef_[0][_id])

пара -3.51471601853
голов -3.49888455332
палец -3.49888455332
меч -3.49366260933
вид -3.4832998223
словно -3.4781584228
помочь -3.4781584228
немного -3.4781584228
голос -3.46289095067
пытаться -3.45785315664
минута -3.45785315664
подумать -3.45284061481
стол -3.45284061481
ждать -3.44289028396
кто -3.43303798752
по -3.42814800222
свет -3.42328181257
случай -3.4088237294
пойти -3.4088237294
женщина -3.4088237294
затем -3.39929984789
хороший -3.38986581565
мысль -3.38986581565
достать -3.38518196634
сидеть -3.38518196634
быстро -3.37126062782
комната -3.35299527984
нет -3.34848059949
понимать -3.33505757916
сердце -3.33062298209
мир -3.32620796388
взгляд -3.32620796388
ответить -3.31743597781
должный -3.31743597781
решить -3.31307867244
никто -3.3044206097
работа -3.3001195278
смерть -3.295836866
смочь -3.295836866
посмотреть -3.295836866
сразу -3.29157246722
шея -3.28732617634
выйти -3.28309784023
ряд -3.28309784023
какой -3.27888730769
земля -3.27469442943
оказаться -3.26636104787
жить

In [361]:
# визуализируем фичаимпортэнсы
import matplotlib.pyplot as plt

importances = clf_bnb.feature_importances_
indices = np.argsort(importances)[::-1]

plt.figure()
plt.title("Feature importances")
plt.bar(range(X_train.shape[1]), importances[indices], color="r", yerr=std[indices], align="center")
plt.xticks(range(X_train.shape[1]), indices)
plt.xlim([-1, X_train.shape[1]])
plt.show()

AttributeError: 'BernoulliNB' object has no attribute 'feature_importances_'